# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.25,37.83,52.0,1279.0,287.0,534.0,291.0,3.1429,231600.0,NEAR BAY
1,-122.25,37.80,41.0,1471.0,469.0,1062.0,413.0,1.6121,171400.0,NEAR BAY
2,-122.21,37.79,35.0,1745.0,409.0,1143.0,386.0,2.8750,143800.0,NEAR BAY
3,-122.19,37.80,52.0,1813.0,271.0,637.0,277.0,4.0114,263400.0,NEAR BAY
4,-122.20,37.78,39.0,1752.0,399.0,1071.0,376.0,3.1167,121600.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
206,-118.86,35.90,38.0,298.0,55.0,161.0,47.0,4.1250,71300.0,INLAND
207,-119.32,36.19,11.0,1281.0,291.0,861.0,313.0,1.0962,72300.0,INLAND
208,-119.06,36.08,19.0,2554.0,443.0,1301.0,419.0,4.1856,72100.0,INLAND
209,-119.03,34.25,25.0,3344.0,502.0,1483.0,496.0,6.1960,340600.0,<1H OCEAN


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StandardScaler, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
train_set = train_set.withColumn('mean_bedrooms_households', train_set['total_bedrooms_'] / train_set['households'])
train_set = train_set.withColumn('mean_rooms_households', train_set['total_rooms'] / train_set['households'])
train_set = train_set.withColumn('mean_population_households', train_set['population'] / train_set['households'])

test_set = test_set.withColumn('mean_bedrooms_households', test_set['total_bedrooms_'] / test_set['households'])
test_set = test_set.withColumn('mean_rooms_households', test_set['total_rooms'] / test_set['households'])
test_set = test_set.withColumn('mean_population_households', test_set['population'] / test_set['households'])

**Etapa ##:** Unir todas as colunas em uma única que será utilizada pelos modelos de Machine Learning do PySpark. Essa etapa dever ser feita antes da padronização dos atributos com StandardScaler.

In [0]:
cols = ['longitude', 'latitude', 'housing_median_age', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_', 'total_bedrooms_', 'mean_bedrooms_households', 'mean_rooms_households', 'mean_population_households']

va = VectorAssembler(inputCols=cols, outputCol='X')
train_set = va.transform(train_set)
test_set  = va.transform(test_set)

**Etapa ##:** Padronização dos atributos com StandardScaler.

In [0]:
ss = StandardScaler(inputCol='X', outputCol='X_')
ss_model = ss.fit(train_set)
train_set = ss_model.transform(train_set)
test_set  = ss_model.transform(test_set)

**Etapa ##:** Selecionar X e y dos conjuntos de treino e teste. Vou renomear "X_" para "X"  e "median_house_value" para "y" assim os conjuntos ficarão de acordo com o padrão de nomenclatura.

In [0]:
train_set = train_set.select('X_', 'median_house_value')
test_set  = test_set.select('X_', 'median_house_value')

train_set = train_set.withColumnRenamed(existing='X_', new='X')
test_set  = test_set.withColumnRenamed(existing='X_', new='X')

train_set = train_set.withColumnRenamed(existing='median_house_value', new='y')
test_set  = test_set.withColumnRenamed(existing='median_house_value', new='y')

# Model training


In [0]:
from pyspark.ml.regression import GBTRegressor, LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

## ElasticNet

In [0]:
lr = LinearRegression(featuresCol='X', labelCol='y', elasticNetParam=0.0)
lr_model = lr.fit(train_set)
y_pred = lr_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show()

mse: 4.909333222250553e-13
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.999450738519...|103600.0|103599.99999979923|
|[-61.984487073819...| 79000.0| 79000.00000072406|
|[-61.974511297353...| 76100.0|  76100.0000014314|
|[-61.954559744420...| 68400.0| 68399.99999980754|
|[-61.939596079721...| 72200.0| 72200.00000050061|
|[-61.939596079721...| 70500.0| 70500.00000053334|
|[-61.934608191488...| 86400.0| 86400.00000073943|
|[-61.934608191488...|128900.0|128900.00000065309|
|[-61.934608191488...|116100.0| 116100.0000006758|
|[-61.934608191488...| 70500.0| 70500.00000040686|
|[-61.929620303254...| 74700.0| 74700.00000034424|
|[-61.929620303254...| 96000.0| 96000.00000044591|
|[-61.924632415021...|104200.0|104200.00000042739|
|[-61.924632415021...| 57500.0| 57500.00000050497|
|[-61.924632415021...|103100.0|103099.99999935117|
|[-61.919644526788...|130600.0|130600.00000052659|
|[-6

## RandomForestRegressor

In [0]:
rf = RandomForestRegressor(featuresCol='X', labelCol='y')
rf_model = rf.fit(train_set)
y_pred = rf_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show()

mse: 469674181.37641764
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.999450738519...|103600.0|117087.31088475874|
|[-61.984487073819...| 79000.0| 90822.40265149243|
|[-61.974511297353...| 76100.0| 89199.55684370897|
|[-61.954559744420...| 68400.0|100193.85188189411|
|[-61.939596079721...| 72200.0| 90631.36708176338|
|[-61.939596079721...| 70500.0| 86789.21121487628|
|[-61.934608191488...| 86400.0|103752.67647707608|
|[-61.934608191488...|128900.0| 149440.5460836324|
|[-61.934608191488...|116100.0|136112.32436052157|
|[-61.934608191488...| 70500.0| 88806.23686771796|
|[-61.929620303254...| 74700.0|   89392.160314791|
|[-61.929620303254...| 96000.0|112228.28777005573|
|[-61.924632415021...|104200.0| 112054.2701827756|
|[-61.924632415021...| 57500.0| 86696.82181819607|
|[-61.924632415021...|103100.0| 96509.40447233846|
|[-61.919644526788...|130600.0|140372.42434680444|
|[-61.9

## GBTRegressor

In [0]:
gb = GBTRegressor(featuresCol='X', labelCol='y')
gb_model = gb.fit(train_set)
y_pred = gb_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show()

mse: 32177527.92472595
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.999450738519...|103600.0|103734.73413440713|
|[-61.984487073819...| 79000.0| 80440.24608033166|
|[-61.974511297353...| 76100.0| 80399.83774754201|
|[-61.954559744420...| 68400.0| 69788.82473940981|
|[-61.939596079721...| 72200.0| 69692.47128053974|
|[-61.939596079721...| 70500.0|  70014.7548013481|
|[-61.934608191488...| 86400.0| 89801.02459705202|
|[-61.934608191488...|128900.0|129341.49935358019|
|[-61.934608191488...|116100.0|118347.08598715598|
|[-61.934608191488...| 70500.0|  69933.5313821201|
|[-61.929620303254...| 74700.0| 70571.77022474671|
|[-61.929620303254...| 96000.0| 95498.33976290248|
|[-61.924632415021...|104200.0|103694.32580161748|
|[-61.924632415021...| 57500.0| 61370.77237593507|
|[-61.924632415021...|103100.0|103342.16618452987|
|[-61.919644526788...|130600.0|128965.18618591306|
|[-61.91